In [1]:
#from pyspark.sql.types import *
#from pyspark.sql.functions import *
#from datetime import datetime

# File location and type
file_location = "/FileStore/tables/planecrashinfo_20181121001952.csv"

# The applied options are for CSV files. For other file types, these will be ignored.
# .option("dateFormat", "MMM d, yy") seems not to work
df = spark.read.format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("sep", ",") \
  .load(file_location)

# Hence date conversion has to be done manually
df2 = df.withColumn("date", to_date(df.date, 'MMM d, yy'))
df2.show()

+----------+-------+--------------------+--------------------+---------+-------------+--------------------+------------+-----+--------------------+--------------------+------+--------------------+
 date| time| location| operator|flight_no| route| ac_type|registration|cn_ln| aboard| fatalities|ground| summary|
+----------+-------+--------------------+--------------------+---------+-------------+--------------------+------------+-----+--------------------+--------------------+------+--------------------+
1908-09-17| 17:18| Fort Myer, Virginia|Military - U.S. Army| ?|Demonstration| Wright Flyer III| ?| 1|2   (passengers:1...|1   (passengers:1...| 0|During a demonstr...|
1909-09-07| ?|Juvisy-sur-Orge, ...| ?| ?| Air show| Wright Byplane| SC1| ?|1   (passengers:0...|1   (passengers:0...| 0|Eugene Lefebvre w...|
1912-07-12| 06:30|Atlantic City, Ne...|Military - U.S. Navy| ?| Test flight| Dirigible| ?| ?|5   (passengers:0...|5   (passengers:0...| 0|First U.S. dirigi...|
1913-08-06| ?|Victoria, British...| Private| ?| ?| Curtiss seaplane| ?| ?|1   (passengers:0...|1   (passengers:0...| 0|The first fatal a...|
1913-09-09|c 18:30| Over the North Sea|Military - German...| ?| ?|Zeppelin L-1 (air...| ?| ?|20   (passengers:...|14   (passengers:...| 0|The airship flew ...|
1913-10-17|c 10:30|Near Johannisthal...|Military - German...| ?| ?|Zeppelin L-2 (air...| ?| ?|30   (passengers:...|30   (passengers:...| 0|Hydrogen gas whic...|
1915-03-05|c 01:00| Tienen, Belgium|Military - German...| ?| ?|Zeppelin L-8 (air...| ?| ?|41   (passengers:...|21   (passengers:...| 0|Crashed into tree...|
1915-09-03|c 15:20|Off Cuxhaven, Ger...|Military - German...| ?| ?|Zeppelin L-10 (ai...| ?| ?|19   (passengers:...|19   (passengers:...| 0|Exploded and burn...|
1916-07-28| ?|Near Jambol, Bulg...|Military - German...| ?| ?|Schutte-Lanz S-L-...| ?| ?|20   (passengers:...|20   (passengers:...| 0|Crashed near the ...|
1916-09-24|c 01:00| Billericay, England|Military - German...| ?| ?|Zeppelin L-32 (ai...| ?| ?|22   (passengers:...|22   (passengers:...| 0|Shot down by Brit...|
1916-10-01|c 23:45|Potters Bar, England|Military - German...| ?| ?|Zeppelin L-31 (ai...| ?| ?|19   (passengers:...|19   (passengers:...| 0|Shot down in flam...|
1916-11-21| ?| Mainz, Germany|Military - German...| ?| ?|Super Zeppelin (a...| ?| ?|28   (passengers:...|27   (passengers:...| 0| Crashed in a storm.|
1916-11-28|c 23:45|Off West Hartlepo...|Military - German...| ?| ?|Zeppelin L-34 (ai...| ?| ?|20   (passengers:...|20   (passengers:...| 0|Shot down by Brit...|
1917-03-04| ?| Near Gent, Belgium|Military - German...| ?| ?| Airship| ?| ?|20   (passengers:...|20   (passengers:...| 0|Caught fire and c...|
1917-03-30| ?|Off Northern Germany|Military - German...| ?| ?|Schutte-Lanz S-L-...| ?| ?|23   (passengers:...|23   (passengers:...| 0|Struck by lightni...|
1917-05-14|c 05:15|Near Texel Island...|Military - German...| ?| ?|Zeppelin L-22 (ai...| ?| ?|21   (passengers:...|21   (passengers:...| 0|Crashed into the ...|
1917-06-14|c 08:45|Off Vlieland Isla...|Military - German...| ?| ?|Zeppelin L-43 (ai...| ?| ?|24   (passengers:...|24   (passengers:...| 0|Shot down by Brit...|
1917-06-17| ?|Near Yarmouth, En...| ?| ?| ?|Zepplin LZ-95 (ai...| L-48| ?|16   (passengers:...|14   (passengers:...| 0|Shot down by airc...|
1917-08-21|c 07:00| Off western Denmark|Military - German...| ?| ?|Zeppelin L-23 (ai...| ?| ?|18   (passengers:...|18   (passengers:...| 0|Shot down by Brit...|
1917-10-20|c 07:45|Near Luneville, F...|Military - German...| ?| ?|Zeppelin L-44 (ai...| ?| ?|18   (passengers:...|18   (passengers:...| 0|Shot down by Fren...|
+----------+-------+--------------------+--------------------+---------+-------------+--------------------+------------+-----+--------------------+--------------------+------+--------------------+
only showing top 20 rows

In [2]:
# Create a temp table on top of the dataframe

df2.registerTempTable("PlaneCrashes")

In [3]:
%sql

--select date, sum(substring_index(aboard, '(', 1)), sum(substring_index(fatalities, '()'))
-- List the 15 most deadly years in aviation 
select year(date)
       ,sum(substring(aboard,1,3))
       ,sum(substring(fatalities,1,3)) 
from PlaneCrashes
group by year(date)
order by sum(substring(fatalities,1,3)) desc
limit 8








year(date),"sum(CAST(substring(aboard, 1, 3) AS DOUBLE))","sum(CAST(substring(fatalities, 1, 3) AS DOUBLE))"
1972,3462.0,2769.0
1985,3321.0,2505.0
1973,3305.0,2282.0
1974,2958.0,2215.0
1996,2900.0,2194.0
1989,3482.0,2106.0
1962,2446.0,2036.0
1968,2789.0,2023.0


In [4]:
%sql

select year(date), ac_type from PlaneCrashes
where operator not like '%Military%'
group by year(date), ac_type
order by year(date) desc




year(date),ac_type
2018,Piper PA-32RT-300T
2018,Junkers JU-52
2018,Boeing 757-N23
2018,Cessna 208B Grand Caravan
2018,Boeing 737-8BK
2018,de Havilland Canada DHC-8-402Q Dash 8
2018,Boeing 737-MAX 8
2018,Iran Aseman Airlines
2018,Convair CV-340
2018,Eurocopter EC 130B4


In [5]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "PlaneCrashes"

df2.write.format("parquet").saveAsTable(permanent_table_name)